# MetaCognition Deep Dive

This notebook explores NeSy-Core's primary innovation:
1. Three-dimensional confidence computation
2. Self-doubt layer
3. Confidence calibration (Platt scaling)
4. Knowledge boundary estimation
5. Reasoning traces

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join('..', '..')))

from nesy.api.nesy_model import NeSyModel
from nesy.core.types import (
    Predicate, SymbolicRule, ConceptEdge,
    NullSet, PresentSet, NullItem, NullType,
    ConfidenceReport, LogicClause, LogicConnective,
    ReasoningStep,
)
from nesy.metacognition.confidence import (
    compute_factual, compute_reasoning, compute_boundary,
    build_confidence_report,
)
from nesy.metacognition.doubt import SelfDoubtLayer
from nesy.metacognition.calibration import ConfidenceCalibrator
from nesy.metacognition.boundary import KNNBoundaryEstimator

## 1. Three-Dimensional Confidence

NeSy-Core computes three independent confidence axes:
- **Factual**: How complete is the evidence? (inverse anomaly)
- **Reasoning**: How strong are the logical derivations?
- **Knowledge Boundary**: Is the query within the model's competence?

In [ ]:
present = PresentSet(concepts={'fever', 'cough'}, context_type='medical')
null_set = NullSet(
    items=[
        NullItem(concept='headache', weight=0.4,
                 null_type=NullType.TYPE2_MEANINGFUL,
                 expected_because_of=['fever']),
    ],
    present_set=present,
)

c_factual = compute_factual(null_set)
print(f'Factual confidence: {c_factual:.3f}')

clauses = [
    LogicClause(predicates=[Predicate('fever')], connective=LogicConnective.AND,
                satisfied=True, weight=0.85),
    LogicClause(predicates=[Predicate('flu_likely')], connective=LogicConnective.AND,
                satisfied=True, weight=0.90),
]
c_reasoning = compute_reasoning(0.85, clauses)
print(f'Reasoning confidence: {c_reasoning:.3f}')

c_boundary = compute_boundary(0.90, null_set)
print(f'Boundary confidence: {c_boundary:.3f}')

## 2. Full Confidence Report

In [ ]:
report = build_confidence_report(
    null_set=null_set,
    symbolic_confidence=0.85,
    neural_confidence=0.90,
    clauses=clauses,
)
print(f'Factual:   {report.factual:.3f}')
print(f'Reasoning: {report.reasoning:.3f}')
print(f'Boundary:  {report.knowledge_boundary:.3f}')
print(f'Minimum:   {report.minimum:.3f}')
print(f'Reliable:  {report.is_reliable}')

## 3. Self-Doubt Layer

Three triggers: low confidence, critical nulls, topological disconnection.

In [ ]:
doubt = SelfDoubtLayer(threshold=0.60, betti_warn=2)
status, flags = doubt.evaluate(confidence=report, null_set=null_set, betti_0=1)
print(f'Status: {status}')
print(f'Flags: {flags}')

# Test with critical null
critical_ns = NullSet(
    items=[NullItem(concept='chest_pain', weight=0.8,
                    null_type=NullType.TYPE3_CRITICAL,
                    expected_because_of=['cardiac'])],
    present_set=present,
)
s2, f2 = doubt.evaluate(confidence=report, null_set=critical_ns, betti_0=1)
print(f'With critical null: {s2}, flags={f2}')

## 4. End-to-End Model with MetaCognition

In [ ]:
model = NeSyModel(domain='medical', strict_mode=True)
model.add_rules([
    SymbolicRule(id='r1', antecedents=[Predicate('fever'), Predicate('cough')],
                 consequents=[Predicate('flu_likely')], weight=0.85),
    SymbolicRule(id='r2', antecedents=[Predicate('flu_likely')],
                 consequents=[Predicate('recommend_rest')], weight=0.90),
])
model.add_concept_edges([
    ConceptEdge(source='fever', target='headache',
                cooccurrence_prob=0.6, causal_strength=0.5, temporal_stability=1.0),
])

output = model.reason(facts={Predicate('fever'), Predicate('cough')}, context_type='medical')
print(f'Answer:  {output.answer}')
print(f'Status:  {output.status}')
print(f'Conf:    {output.confidence.minimum:.3f}')
print(f'Flags:   {output.flags}')

## 5. Confidence Calibration (Platt Scaling)

In [ ]:
import random
random.seed(42)
cal = ConfidenceCalibrator(n_bins=10)
data = [(random.uniform(0.3, 1.0), random.random() < random.uniform(0.3, 1.0)) for _ in range(200)]
cal.fit(data)
print(f'Raw 0.75 -> Calibrated {cal.calibrate(0.75):.3f}')
print(f'ECE: {cal.ece:.4f}')

## 6. Knowledge Boundary Estimation

In [ ]:
est = KNNBoundaryEstimator(k=3, threshold=0.50)
est.fit([[1,0,0,0],[0.9,0.1,0,0],[0.8,0.2,0.1,0],[1,0.1,0.1,0]])

score_in = est.estimate([0.95, 0.05, 0, 0])
print(f'In-dist score: {score_in:.3f}, in_dist={est.is_in_distribution([0.95,0.05,0,0])}')

score_out = est.estimate([0, 0, 0, 5])
print(f'OOD score: {score_out:.3f}, in_dist={est.is_in_distribution([0,0,0,5])}')

## Summary

NeSy-Core metacognition provides:
- 3D Confidence (Factual + Reasoning + Boundary)
- Self-Doubt with automatic output withholding
- Platt scaling calibration with ECE
- KNN boundary estimation
- Full auditable ReasoningTrace